## Dataset Collection

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
f_mnist = keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = f_mnist.load_data()

In [ ]:
x_train_full[0]

In [ ]:
y_train_full[0]

In [ ]:
x_valid, x_train = x_train_full[:5000]/255, x_train_full[5000:]/255
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
x_test = x_test/255

In [ ]:
x_train.shape

In [ ]:
x_train = x_train[..., np.newaxis]
x_valid = x_valid[..., np.newaxis]
x_test = x_test[..., np.newaxis]

In [ ]:
x_train.shape

In [ ]:
classes = ["T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

## Data Visualization

In [ ]:
plt.imshow(x_train[96], cmap='binary')
plt.show()

In [ ]:
classes[y_train[96]]

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols*1.2, n_rows*1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols*row + col
        plt.subplot(n_rows, n_cols, index+1)
        plt.imshow(x_train[index], cmap='binary')
        plt.axis('off')
        plt.title(classes[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

## Convolutional Neural Networks(CNN)

In [ ]:
model = keras.models.Sequential([
    #input layer
    keras.layers.Conv2D(filters=64, kernel_size=7, activation='relu', padding='same',
                       input_shape=[28, 28, 1]),
    #hidden layer
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    #output layer
    keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd',
             metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid))

## confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
conf_mx = confusion_matrix(y_test, y_pred)
conf_mx

In [ ]:
accuracy_score(y_test, y_pred)

## Image PreProcessing

In [ ]:
import cv2

def predict_class():
    file = file_path
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    resize_img = cv2.resize(img, (28,28), interpolation = cv2.INTER_LINEAR)
    final_img = cv2.bitwise_not(resize_img)
    final_img = final_img/255
    final_img = final_img[..., np.newaxis]
    pred = model.predict(final_img)
    class_name = classes[np.argmax(pred)]
    label = Label(window, text=class_name, font=('Bodoni MT Black', 15), padx=40, pady=20)
    label.grid(column=0, row=3)

## Tkinter

In [ ]:
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog

window = Tk()
window.title("Clothing Ramification")
window.geometry("750x450")
window.minsize(750, 550)
window.maxsize(750, 550)
window.config(bg='black')
file_path = ''
def access_image():
    global file_path
    file_path = selected_file()
    
    img = Image.open(file_path)
    img = img.resize((300, 300))
    img = ImageTk.PhotoImage(img)
    label= Label(window, image=img, padx=300)
    label.photo = img
    label.grid(column=0, row=1)
    
def selected_file():
    filename = filedialog.askopenfilename(initialdir='H:\Machine Learning\Image_for_task',
                                         title='Select a File',
                                         filetype = (('JPG File', "*.jpg*"),
                                                    ('all files', "*.*")))
    return filename
    
button1 = Button(window, text='Select Image', padx=30,fg='black', bg='gray', command=access_image)
button1.grid(row=0, column=0, padx=300, pady=25)

img = Image.open(r"H:\Machine Learning\Tkinter\init.jpg")
img = img.resize((300, 300))
img = ImageTk.PhotoImage(img)
img_label= Label(window, image=img)
img_label.grid(column=0, row=1)

button2 = Button(window, text='Predict', padx=30, fg='black', bg='gray', command=predict_class)
button2.grid(row=2, column=0, pady=25)
window.mainloop()